## import Libraries

In [3]:
import os
import numpy as np
from PIL import Image
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
import matplotlib.pyplot as plt

## Load and Preprocess Data

In [5]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

def load_and_preprocess_data(data_dirs, img_size, batch_size, validation_split):
    train_datagens = []
    val_datagens = []
    train_generators = []
    val_generators = []

    for data_dir in data_dirs:
        train_datagen = ImageDataGenerator(
            rescale=1./255,
            validation_split=validation_split,
            rotation_range=40,
            width_shift_range=0.2,
            height_shift_range=0.2,
            shear_range=0.2,
            zoom_range=0.2,
            horizontal_flip=True,
            fill_mode='nearest'
        )

        train_generator = train_datagen.flow_from_directory(
            data_dir,
            target_size=(img_size, img_size),
            batch_size=batch_size,
            class_mode='categorical',
            subset='training',
            shuffle=True
        )

        val_generator = train_datagen.flow_from_directory(
            data_dir,
            target_size=(img_size, img_size),
            batch_size=batch_size,
            class_mode='categorical',
            subset='validation',
            shuffle=False
        )

        train_datagens.append(train_datagen)
        train_generators.append(train_generator)
        val_generators.append(val_generator)

    return train_generators, val_generators



In [7]:
base_data_dir = "TrashType_Image_Dataset"
categories = ["cardboard", "glass", "metal", "paper", "plastic", "trash"]
data_dirs = [os.path.join(base_data_dir, category) for category in categories]
img_size = 224
batch_size = 32
validation_split = 0.2

train_generators, val_generators = load_and_preprocess_data(data_dirs, img_size, batch_size, validation_split)

Found 0 images belonging to 0 classes.


Found 0 images belonging to 0 classes.
Found 0 images belonging to 0 classes.
Found 0 images belonging to 0 classes.
Found 0 images belonging to 0 classes.
Found 0 images belonging to 0 classes.
Found 0 images belonging to 0 classes.
Found 0 images belonging to 0 classes.
Found 0 images belonging to 0 classes.
Found 0 images belonging to 0 classes.
Found 0 images belonging to 0 classes.
Found 0 images belonging to 0 classes.
